In [1]:
# Standard libraries
import os
from glob import glob
import yaml

# Numerical computation and image processing
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# PyTorch and deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data                  import Dataset, DataLoader
from torch.utils.tensorboard           import SummaryWriter
from torch.optim.lr_scheduler          import ReduceLROnPlateau
from torchvision                       import models, transforms, utils
from torchvision.models                import VGG16_Weights
from torchvision.utils                 import save_image, make_grid
from torchvision.transforms.functional import to_pil_image

# Evaluation metrics
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

# Utilities
from tqdm import tqdm

# Set multiprocessing method
import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)

# Run Python script
import sys
sys.path.append('/kaggle/input/python-files')

from InitDataset import InitDataset
from MaskGenerator import MaskGenerator

# **I - Generate Dataset**
---

In [ ]:
# class MaskGenerator(object):
#     def __init__(self, height, width, channels=3, num_irregulars=10):
#         self.height         = height
#         self.width          = width
#         self.channels       = channels
#         self.num_irregulars = np.random.randint(1, num_irregulars + 1)

#     def generate(self):
#         mask = np.zeros((self.height, self.width, self.channels), np.uint8)
#         size = int((self.width + self.height) * 0.03)

#         for _ in range(self.num_irregulars):
#             x1, x2    = np.random.randint(1, self.width , size=2)
#             y1, y2    = np.random.randint(1, self.height, size=2)
#             thickness = np.random.randint(3, size)
#             cv2.line(mask, (x1, y1), (x2, y2), (1, 1, 1), thickness)

#         for _ in range(self.num_irregulars):
#             x1, y1 = np.random.randint(1, self.width), np.random.randint(1, self.height)
#             radius = np.random.randint(3, size)
#             cv2.circle(mask, (x1, y1), radius, (1, 1, 1), -1)

#         for _ in range(self.num_irregulars):
#             x1, y1      = np.random.randint(1, self.width)     , np.random.randint(1, self.height)
#             s1, s2      = np.random.randint(1, self.width // 4), np.random.randint(1, self.height // 4)
#             angle       = np.random.randint(0, 180)
#             start_angle = np.random.randint(0, 180)
#             end_angle   = np.random.randint(start_angle, 180)
#             thickness   = np.random.randint(3, size)
#             cv2.ellipse(mask, (x1, y1), (s1, s2), angle, start_angle, end_angle, (1, 1, 1), thickness)

#         # Require: 0 = area to restore, 1 = area not affected
#         return 1 - mask

# # Testing
# maskGenerator = MaskGenerator(512, 512)
# _, axes = plt.subplots(2, 5, figsize=(20, 8))
# for ax in axes.ravel():
#     mask = maskGenerator.generate() * 255
#     ax.imshow(mask)
#     ax.set_xticks([])
#     ax.set_yticks([])

In [ ]:
# class InitDataset(Dataset):
#     def __init__(self, data_root, img_transform, mask_transform, data='train', maskGenerator=None):
#         super().__init__()
#         self.img_transform  = img_transform
#         self.mask_transform = mask_transform
#         self.maskGenerator  = maskGenerator 
#         self.paths          = glob(f'{data_root}/COCO2017/{data}2017/*')
        
#     def __len__(self):
#         return len(self.paths)

#     def __getitem__(self, index):
#         img = self.load_img(self.paths[index])
#         img = self.img_transform(img.convert('RGB'))
        
#         mask = self.maskGenerator.generate() * 255
#         mask = Image.fromarray(mask.astype(np.uint8))
#         mask = self.mask_transform(mask.convert('RGB'))

#         return img * mask, mask, img

#     def load_img(self, path):
#         try:
#             img = Image.open(path)
#             return img
#         except (FileNotFoundError, UnidentifiedImageError):
#             raise FileNotFoundError(f"x Can't load image from: {path}")

# Testing
# dataset_train = InitDataset(
#     '/kaggle/input/coco2017-dataset', 
#     transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()]), 
#     transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()]), 
#     data='train',
#     maskGenerator = MaskGenerator(256, 256, channels=3)
# )
# _, axes = plt.subplots(1, 3, figsize=(20, 4))
# for ax, ele in zip(axes, dataset_train[0]):
#     ax.imshow(to_pil_image(ele))
#     ax.set_xticks([])
#     ax.set_yticks([])

# **II - Define the Model**
---

In [4]:
class PartialConvolution(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
        super().__init__(in_channels, out_channels, kernel_size, 
                         stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias, padding_mode=padding_mode)
        
        # Kernel for updating mask
        self.mask_kernel = torch.ones(self.out_channels, self.in_channels, self.kernel_size[0], self.kernel_size[1])
        self.register_buffer('mask_kernel_buffer', self.mask_kernel) 

        # For renormalization
        self.sum1 = self.mask_kernel.shape[1] * self.mask_kernel.shape[2] * self.mask_kernel.shape[3]
        
        # Initialize weights for image convolution
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, img, mask):
        mask_kernel = self.mask_kernel_buffer

        # Calculate update mask
        update_mask = F.conv2d(mask, mask_kernel, bias=None, stride=self.stride, padding=self.padding, dilation=self.dilation, groups=1)
        mask_ratio  = self.sum1 / (update_mask + 1e-8)
        update_mask = torch.clamp(update_mask, 0, 1)
        mask_ratio  = mask_ratio * update_mask

        # Apply partial convolution
        conved = F.conv2d(img * mask, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)

        # Apply mask ratio normalization
        if self.bias is not None:
            bias_view = self.bias.view(1, self.out_channels, 1, 1)
            output    = mask_ratio * (conved - bias_view) + bias_view
        else:
            output = conved * mask_ratio

        return output, update_mask

class UpsampleData(nn.Module):
    def __init__(self):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, dec_feature, enc_feature, dec_mask, enc_mask):
        out      = torch.cat([self.upsample(dec_feature), enc_feature], dim=1)
        out_mask = torch.cat([self.upsample(dec_mask)   , enc_mask]   , dim=1)
        return out, out_mask

class PConvLayer(nn.Module):
    def __init__(self, in_ch, out_ch, sample='none-3', dec=False, bn=True, active='relu', conv_bias=False):
        super().__init__()
        
        kernel_map = {'down-7': (7, 2, 3), 'down-5': (5, 2, 2), 'down-3': (3, 2, 1)}
        kernel_size, stride, padding = kernel_map.get(sample, (3, 1, 1))
        
        self.conv = PartialConvolution(in_ch, out_ch, kernel_size, stride, padding, bias=conv_bias)
        if dec:
            self.upcat = UpsampleData()
        if bn:
            self.bn = nn.BatchNorm2d(out_ch)
        if active == 'relu':
            self.activation = nn.ReLU()
        elif active == 'leaky':
            self.activation = nn.LeakyReLU(negative_slope=0.2)

    def forward(self, img, mask, enc_img=None, enc_mask=None):
        if hasattr(self, 'upcat'):
            out, update_mask = self.upcat(img, enc_img, mask, enc_mask)
            out, update_mask = self.conv(out, update_mask)
        else:
            out, update_mask = self.conv(img, mask)
            
        if hasattr(self, 'bn'):
            out = self.bn(out)
        if hasattr(self, 'activation'):
            out = self.activation(out)
            
        return out, update_mask

In [ ]:
class PConvUNet(nn.Module):
    def __init__(self, finetune=False, in_ch=3, layer_size=6):
        super().__init__()
        self.freeze_enc_bn = True if finetune else False
        self.layer_size    = layer_size

        self.enc_1 = PConvLayer(in_ch, 64, 'down-7', bn=False)
        self.enc_2 = PConvLayer(64,   128, 'down-5')
        self.enc_3 = PConvLayer(128,  256, 'down-5')
        self.enc_4 = PConvLayer(256,  512, 'down-3')
        self.enc_5 = PConvLayer(512,  512, 'down-3')
        self.enc_6 = PConvLayer(512,  512, 'down-3')
        self.enc_7 = PConvLayer(512,  512, 'down-3')
        self.enc_8 = PConvLayer(512,  512, 'down-3')

        self.dec_8 = PConvLayer(512 + 512, 512, dec=True, active='leaky')
        self.dec_7 = PConvLayer(512 + 512, 512, dec=True, active='leaky')
        self.dec_6 = PConvLayer(512 + 512, 512, dec=True, active='leaky')
        self.dec_5 = PConvLayer(512 + 512, 512, dec=True, active='leaky')
        self.dec_4 = PConvLayer(512 + 256, 256, dec=True, active='leaky')
        self.dec_3 = PConvLayer(256 + 128, 128, dec=True, active='leaky')
        self.dec_2 = PConvLayer(128 + 64,   64, dec=True, active='leaky')
        self.dec_1 = PConvLayer(64  + 3,     3, dec=True, bn=False, active=None, conv_bias=True)

    def forward(self, img, mask):
        enc_f, enc_m = [img], [mask]
        
        for layer_num in range(1, self.layer_size + 1):
            if layer_num == 1:
                feature, update_mask = getattr(self, f'enc_{layer_num}')(img, mask)
            else:
                enc_f.append(feature)
                enc_m.append(update_mask)
                feature, update_mask = getattr(self, f'enc_{layer_num}')(feature, update_mask)

        assert len(enc_f) == self.layer_size

        for layer_num in reversed(range(1, self.layer_size + 1)):
            feature, update_mask = getattr(self, f'dec_{layer_num}')(feature, update_mask, enc_f.pop(), enc_m.pop())

        return feature, mask

    def train(self, mode=True):
        super().train(mode)
        if self.freeze_enc_bn:
            for name, module in self.named_modules():
                if isinstance(module, nn.BatchNorm2d) and 'enc' in name:
                    module.eval()

# # Testing
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model  = PConvUNet(in_ch=3, layer_size=6).to(device)
# input_image = torch.rand(4, 3, 256, 256).to(device)
# input_mask  = torch.zeros_like(input_image).to(device)
# input_mask[:, :, 64:192, 64:192] = 1
# model.eval()
# with torch.no_grad():
#     output_image, output_mask = model(input_image, input_mask)
# print(shapes[0]) if all(s == shapes[0] for s in shapes) else print(shapes)

# **III - Define the Loss Function**
---

In [6]:
class InpaintingLoss(nn.Module):
    def __init__(self, extractor, tv_loss='mean'):
        super().__init__()
        self.tv_loss   = tv_loss
        self.l1        = nn.L1Loss()
        self.extractor = extractor
 
    def forward(self, input, mask, output, gt):
        comp = mask * input + (1 - mask) * output

        tv_loss    = total_variation_loss(comp, mask, self.tv_loss)
        hole_loss  = self.l1((1 - mask) * output, (1 - mask) * gt)
        valid_loss = self.l1(mask * output, mask * gt)

        feats_out  = self.extractor(output)
        feats_comp = self.extractor(comp)
        feats_gt   = self.extractor(gt)
        perc_loss  = 0.0
        style_loss = 0.0

        for i in range(3):
            perc_loss  += self.l1(feats_out[i], feats_gt[i])
            perc_loss  += self.l1(feats_comp[i], feats_gt[i])
            style_loss += self.l1(gram_matrix(feats_out[i]) , gram_matrix(feats_gt[i]))
            style_loss += self.l1(gram_matrix(feats_comp[i]), gram_matrix(feats_gt[i]))

        return {'valid': valid_loss, 'hole': hole_loss, 'perc': perc_loss, 'style': style_loss, 'tv': tv_loss}

In [7]:
class VGG16FeatureExtractor(nn.Module):
    MEAN = [0.485, 0.456, 0.406]
    STD  = [0.229, 0.224, 0.225]

    def __init__(self):
        super().__init__()
        vgg16         = models.vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        normalization = Normalization(self.MEAN, self.STD)
        
        # Feature extractor layers
        self.enc_1 = nn.Sequential(normalization, *vgg16.features[:5])
        self.enc_2 = nn.Sequential(*vgg16.features[5:10])
        self.enc_3 = nn.Sequential(*vgg16.features[10:17])

        # Freeze all VGG layers
        for enc_layer in [self.enc_1, self.enc_2, self.enc_3]:
            for param in enc_layer.parameters():
                param.requires_grad = False

    def forward(self, input):
        feature_maps = [input]
        for i in range(3):
            feature_map = getattr(self, f'enc_{i+1}')(feature_maps[-1])
            feature_maps.append(feature_map)
        return feature_maps[1:]

class Normalization(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std  = torch.tensor(std).view(-1, 1, 1)

    def forward(self, input):
        if self.mean.type() != input.type():
            self.mean = self.mean.to(input)
            self.std  = self.std.to(input)
        return (input - self.mean) / self.std

In [8]:
def gram_matrix(feat):
    (b, ch, h, w) = feat.size()
    feat   = feat.view(b, ch, h * w)
    feat_t = feat.transpose(1, 2)
    gram   = torch.bmm(feat, feat_t) / (ch * h * w)
    return gram

def dialation_holes(hole_mask):
    b, ch, h, w   = hole_mask.shape
    dilation_conv = nn.Conv2d(ch, ch, 3, padding=1, bias=False).to(hole_mask)
    torch.nn.init.constant_(dilation_conv.weight, 1.0)
    
    with torch.no_grad():
        output_mask = dilation_conv(hole_mask)
    updated_holes = output_mask != 0
    return updated_holes.float()

def total_variation_loss(image, mask, method):
    hole_mask       = 1 - mask
    dilated_holes   = dialation_holes(hole_mask)
    colomns_in_Pset = dilated_holes[:, :, :, 1:] * dilated_holes[:, :, :, :-1]
    rows_in_Pset    = dilated_holes[:, :, 1:, :] * dilated_holes[:, :, :-1:, :]
    
    if method == 'sum':
        loss = torch.sum(torch.abs(colomns_in_Pset*(image[:, :, :, 1:] - image[:, :, :, :-1]))) + \
               torch.sum(torch.abs(rows_in_Pset   *(image[:, :, :1, :] - image[:, :, -1:, :])))
    else:
        loss = torch.mean(torch.abs(colomns_in_Pset*(image[:, :, :, 1:] - image[:, :, :, :-1]))) + \
               torch.mean(torch.abs(rows_in_Pset   *(image[:, :, :1, :] - image[:, :, -1:, :])))
    return loss

# **IV - Utility Functions**
---

In [9]:
class Config(dict):
    def __init__(self, conf_file):
        with open(conf_file, "r") as f:
            self._conf = yaml.safe_load(f)

    def __getattr__(self, name):
        return self._conf.get(name)

In [10]:
def load_ckpt(ckpt_path, models, optimizers=None, for_predict=False):
    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model'])

    if for_predict:
        model.eval()
        return model
    else:
        if optimizer and 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
        return checkpoint.get('step', 0)

# **V - Training**
---

In [11]:
class Trainer:
    def __init__(self, step, config, device, model, dataset_train, dataset_val, criterion, optimizer):
        self.stepped          = step
        self.config           = config
        self.device           = device
        self.model            = model
        self.dataloader_train = DataLoader(dataset_train, batch_size=config.batch_size, shuffle=True, num_workers=4, pin_memory=True)
        self.dataset_val      = dataset_val
        self.criterion        = criterion
        self.optimizer        = optimizer
        self.writer           = SummaryWriter(log_dir=config.ckpt)
        self.best_loss        = float('inf')
        #self.scheduler        = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

    def iterate(self):
        print("Training ...")
        torch.autograd.set_detect_anomaly(True)
        progress_bar = tqdm(total=self.config.max_iter, desc="Progress", unit="step")

        for step, (input, mask, gt) in enumerate(self.dataloader_train):
            current_step = step + self.stepped
            loss_dict    = self.train(current_step, input, mask, gt)

            if current_step % self.config.log_interval == 0:
                self.report(current_step, loss_dict)
            if current_step % self.config.vis_interval == 0:
                self.evaluate(current_step)
            if current_step % self.config.save_model_interval == 0:
                self.checkpoint(current_step)
                
            if step >= self.config.max_iter:
                print("- Max iterations reached. Stop training!")
                break

            progress_bar.update(1)

        progress_bar.close()

    def train(self, step, input, mask, gt):
        self.model.train()
        
        input = input.to(self.device)
        mask  = mask.to(self.device)
        gt    = gt.to(self.device)

        output, _ = self.model(input, mask)
        loss_dict = self.criterion(input, mask, output, gt)
        loss      = sum(getattr(self.config, f"{key}_coef") * val for key, val in loss_dict.items())

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update scheduler
        # self.scheduler.step(loss.item())
        
        loss_dict['total'] = loss
        return {k: v.item() for k, v in loss_dict.items()}
        
    def report(self, step, loss_dict):
        print(f"[STEP: {step}] "
              f"\t| Valid Loss: {loss_dict['valid']:.6f} "
              f"\t| Hole Loss: {loss_dict['hole']:.6f} " 
              f"\t| Total Loss: {loss_dict['total']:.6f}")
        
        for key, val in loss_dict.items():
            self.writer.add_scalar(f'Loss/{key}', val, step)
        
    def evaluate(self, step):
        print(f"- Evaluation at [STEP: {step}] ...")
        
        self.model.eval()
        samples         = [self.dataset_val[i] for i in range(8)]
        image, mask, gt = zip(*samples)

        image = torch.stack(image).to(self.device)
        mask  = torch.stack(mask).to(self.device)
        gt    = torch.stack(gt).to(self.device)

        with torch.no_grad():
            output, _ = self.model(image, mask)
        output_comp = mask * image + (1 - mask) * output

        # Tính PSNR và SSIM
        psnr_total, ssim_total = 0, 0
        for i in range(image.size(0)):
            pred_img = output_comp[i].cpu().numpy().transpose(1, 2, 0)
            gt_img   = gt[i].cpu().numpy().transpose(1, 2, 0)

            psnr_total += peak_signal_noise_ratio(gt_img, pred_img, data_range=1.0)
            ssim_total += structural_similarity(gt_img, pred_img, data_range=1.0, multichannel=True, win_size=3)
        avg_psnr = psnr_total / image.size(0)
        avg_ssim = ssim_total / image.size(0)
        print(f"> Average PSNR: {avg_psnr:.6f}, SSIM: {avg_ssim:.6f}")
    
        # Log PSNR và SSIM vào TensorBoard
        self.writer.add_scalar('Validation/PSNR', avg_psnr, step)
        self.writer.add_scalar('Validation/SSIM', avg_ssim, step)

        # Lưu kết quả visualize
        grid = make_grid(torch.cat([image.cpu(), mask.cpu(), output.cpu(), output_comp.cpu(), gt.cpu()], dim=0), nrow=8, padding=2)
        save_image(grid, f'{self.config.ckpt}/val_vis/{step}.png')

        # Tính loss validation
        val_loss_dict = self.criterion(image, mask, output, gt)
        val_loss      = sum(getattr(self.config, f"{key}_coef") * val for key, val in val_loss_dict.items()).item()    
        print(f"> Total Loss: {val_loss:.6f}")
    
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            torch.save({
                'step': step,
                'model': self.model.state_dict(),
                'optimizer': self.optimizer.state_dict(),
                'best_loss': self.best_loss,
            }, f"{self.config.ckpt}/models/best_model.pth")
            print(f"> New best model saved")

    def checkpoint(self, step):
        print(f"- Checkpoint at [STEP: {step}] ...")

        torch.save({
            'step'     : step,
            'model'    : self.model.state_dict(),
            'optimizer': self.optimizer.state_dict()
        }, f"{self.config.ckpt}/models/{step}.pth")

In [12]:
print("Configuring ...")

config      = Config('/kaggle/input/config-files/config (batch 4 - iter 100k vis 2k model 10k log 100).yml')
config.ckpt = "ckpt"
os.makedirs(os.path.join(config.ckpt, "val_vis"), exist_ok=True)
os.makedirs(os.path.join(config.ckpt, "models"), exist_ok=True)

Configuring ...


In [13]:
print("Setup device ...")

device = torch.device(f"cuda:{config.cuda_id}" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = PConvUNet(finetune=config.finetune, layer_size=config.layer_size)

if config.finetune:
    model.load_state_dict(torch.load(config.finetune)['model'])
    
model.to(device)

Setup device ...
Using device: cuda:0


PConvUNet(
  (enc_1): PConvLayer(
    (conv): PartialConvolution(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (activation): ReLU()
  )
  (enc_2): PConvLayer(
    (conv): PartialConvolution(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
  (enc_3): PConvLayer(
    (conv): PartialConvolution(128, 256, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
  (enc_4): PConvLayer(
    (conv): PartialConvolution(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation): ReLU()
  )
  (enc_5): PConvLayer(
    (conv): PartialConvolution(512, 512, kernel_size=(3, 3), stride=(2, 2

In [14]:
img_tf  = transforms.Compose([transforms.Resize((256, 256)),
                              transforms.RandomHorizontalFlip(),
                              transforms.RandomRotation(10),
                              transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                              transforms.ToTensor()])
mask_tf = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

maskGenerator = MaskGenerator(256, 256)

trainer = Trainer(
    step   = 0,
    config = config,
    device = device,
    model  = model,
    
    dataset_train = InitDataset(config.data_root, img_tf, mask_tf, data="train", maskGenerator=maskGenerator),
    dataset_val   = InitDataset(config.data_root, img_tf, mask_tf, data="val"  , maskGenerator=maskGenerator),
    
    criterion = InpaintingLoss(VGG16FeatureExtractor(), tv_loss=config.tv_loss).to(device),
    
    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr           = config.finetune_lr if config.finetune else config.initial_lr,
        weight_decay = config.weight_decay
    )
)
trainer.iterate()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:03<00:00, 142MB/s]


Training ...


Progress:   0%|          | 0/100000 [00:00<?, ?step/s]

[STEP: 0] 	| Valid Loss: 0.403463 	| Hole Loss: 0.210676 	| Total Loss: 834.417297
- Evaluation at [STEP: 0] ...
> Average PSNR: 12.496291, SSIM: 0.734153
> Total Loss: 41.721470
> New best model saved
- Checkpoint at [STEP: 0] ...


Progress:   0%|          | 101/100000 [00:39<7:00:09,  3.96step/s]

...

Progress:  29%|██▉       | 29401/100000 [2:15:30<5:22:29,  3.65step/s]

[STEP: 29400] 	| Valid Loss: 0.016325 	| Hole Loss: 0.025308 	| Total Loss: 3.502898


Progress:  30%|██▉       | 29501/100000 [2:15:57<5:31:28,  3.54step/s]

[STEP: 29500] 	| Valid Loss: 0.015734 	| Hole Loss: 0.021427 	| Total Loss: 3.275001


Progress:  30%|██▉       | 29572/100000 [2:16:19<5:24:39,  3.62step/s]


# **VI - Testing**
---

In [ ]:
# def predict(model, image, mask, device):
#     # Đưa dữ liệu lên thiết bị
#     image, mask = image.to(device), mask.to(device)

#     with torch.no_grad():  # Không cần tính gradient khi dự đoán
#         output, _ = model(image.unsqueeze(0), mask.unsqueeze(0))  # Thêm batch dimension
#         output = output.squeeze(0)  # Loại bỏ batch dimension

#     return output

# # Ví dụ sử dụng
# image = transforms.ToTensor()(Image.open("/path/to/input_image.jpg"))
# mask = transforms.ToTensor()(Image.open("/path/to/mask_image.jpg"))

# reconstructed_image = predict(model, image, mask, device)

# # Hiển thị ảnh kết quả
# reconstructed_image_pil = to_pil_image(reconstructed_image.cpu())
# reconstructed_image_pil.show()